Score:
Accepts a set of motifs and calculates the total distance from the consensus motif.
Total Distance = t*k - Consensus score, where t is the number of sequences, and k is the length of motif
@param sample: Set of motifs, Array{String,1}
@param k: length of motif, Int64

In [1]:
function Score(sample, k)
    t = length(sample)
    score = 0
    for i=1:k
        count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
        for j=1:t
            count[sample[j][i]] = count[sample[j][i]] + 1
        end
        max = 0
        for each in count
            if each[2]>max
                max = each[2]
            end
        end
        score = score+max
    end
    return k*t - score
end

Score (generic function with 1 method)

Make_Profile:
Creates a profile matrix for given set of motifs.
Enabling pseudocounts calculates the profile using Laplace's rule of succession as described in the textbook, page no.88
@param k: motif length, Int64
@param t: number of motifs, Int 64
@param k_mers: Set of motifs, Array{String, 1}
@param pseudocounts: enable pseudocounts, Bool
                   true: uses succession rule
                   false: traditional profile

In [2]:
function Make_Profile(k, t, k_mers, pseudocounts = false)
    pseudo = 0
    if pseudocounts
        pseudo = 1
    end
    count_matrix = zeros(Int16, 4, k)
    for i in 1:k
        for j in 1:t
            c = k_mers[j][i]
            if c == 'A'
                count_matrix[1,i] = count_matrix[1,i] + 1
            end
            if c == 'C'
                count_matrix[2,i] = count_matrix[2,i] + 1
            end
            if c == 'G'
                count_matrix[3,i] = count_matrix[3,i] + 1
            end
            if c == 'T'
                count_matrix[4,i] = count_matrix[4,i] + 1
            end
        end
    end
    profile_matrix = zeros(Float64, 4, k)
    for i in 1:4
        for j in 1:k
            profile_matrix[i,j] = (count_matrix[i,j] + 1*pseudo)/(t + pseudo*4)
        end
    end
    return profile_matrix
end

Make_Profile (generic function with 2 methods)

Get_Kmer:
Returns new motif from the given sequence of DNA string
Calculates the propability score for each possible motif from given string.
Returns the k_mer with highest probability score if deterministic is enabled
Returns a randomly chosen k_mer otherwise. The k_mer is chosen by rolling an unfair die, the higher the profile-probabilty of the k_mer, the higer probabilty of it being chosen.
@param k: length of motif, Int64
@param profile: Motif profile, Array{Float64, 4, t}
@param DNAString: Set of input DNA sequences, Array{String, 1}
@param deterministic: enables deterministic
                    true: returns k_mer with highest profile-probability
                    false: reutrns k_mer using profile-probability as weights for the unfair die

In [3]:
function Get_Kmer(k, profile, DNAString, deterministic)
    n = length(DNAString)
    probArray = zeros(Float64, 1, n-k)
    pdfArray = zeros(Float64, 1, n-k)
    probSum = 0
    for i in 1:n-k
        temp = DNAString[i:i+k-1]
        tempArray = zeros(Int64, 1, k)
        prob = 1.0
        for j in 1:k
            if temp[j] == 'A'
                tempArray[j] = 1
            end
            if temp[j] == 'C'
                tempArray[j] = 2
            end
            if temp[j] == 'G'
                tempArray[j] = 3
            end
            if temp[j] == 'T'
                tempArray[j] = 4
            end
            prob = prob * profile[tempArray[j], j]
        end
        probArray[i] = prob
        probSum = probSum + prob
    end

    probArray[1] = probArray[1] / probSum
    pdfArray[1] = probArray[1]
    for i in 2:length(probArray)
        probArray[i] = probArray[i] / probSum
        pdfArray[i] = pdfArray[i-1] + probArray[i]
    end
    s=findmax(probArray)[1]
    if (deterministic == true)
        for i in 1:length(probArray)
            if s == probArray[i]
                println(i)
                return(DNAString[i:i+k-1])
            end
        end
    end
    p = rand()
    final = 1
    for i in 1:length(pdfArray)
        if (p < pdfArray[i])
            final = i
            break
        end
    end
    #println(p)
    #println(pdfArray)
    #println(final)
    return(DNAString[final:final+k-1])
end

Get_Kmer (generic function with 1 method)

GibbsSampler:
Gibbs sampler to find motif of given length from given set of DNA sequences
Uses Make_Profile function with 'pseudocounts' enabled to create profile
Uses Get_Kmer function with 'deterministic' disabled to select a profile-randomly generated motif from selected sequence.
Uses Score function to calculate total distance for a set of motifs and choose the better one.
@param Dna: Set of input DNA sequences, Array{String, 1}
@param k: length of motif, Int64
@param t: number of sequences, Int64
@param N: Number of trials to be conducted, Int64

In [4]:
function GibbsSampler(Dna, k, t, N)
    n = length(Dna[1])
    s = rand(t)*(n-k+1)
    sTemp = zeros(Int64, 1, t)
    k_mers = Array{String,1}(undef, t)
    for i in 1:t
        s[i] = s[i] + 1
        sTemp[i] = trunc(Int64, s[i])
        k_mers[i] = Dna[i][sTemp[i]:sTemp[i]+k-1]
    end
    BestMotifs = k_mers
    for j in 1:N
        i = trunc(Int64, rand()*t) + 1
        tempMotifs = copy(BestMotifs)
        Motifs = copy(tempMotifs)
        Motifs = deleteat!(Motifs, i)
        profile = Make_Profile(k, t - 1, k_mers, true)
        newMotifAtI = Get_Kmer(k, profile, Dna[i], false)
        tempMotifs[i] = newMotifAtI
        score1 = Score(BestMotifs, k)
        score2 = Score(tempMotifs, k)
        if score2 < score1
            BestMotifs = copy(tempMotifs)
        end
    end
    return BestMotifs
end

GibbsSampler (generic function with 1 method)

Reads length of motif, number of sequences and the DNA sequences from the file "test.txt"
Assumptions: 
1. All DNA sequences are of same length.
2. Length of motif is less than length of each DNA sequence.
3. number of sequnces is more than 1.

In [11]:
open("test.txt") do file
    x = readline(file)
    line1 = split(x," ")
    k = parse(Int64, line1[1])
    t = parse(Int64, line1[2])
    N = parse(Int64, line1[3])
    DNA = Array{String,1}(undef,t)
    for i in 1:t
        DNA[i] = readline(file)
    end
    Motif = GibbsSampler(DNA, k, t, N)
    for each in Motif
        println(each)
    end
    println(DNA)
    println(k)
    println(t)
    println(Score(Motif, k))
end

GCCCCTCT
GCCAAGGT
ATACAGGC
GATCAAGT
CACCAGCT
["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA", "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG", "TAGTACCGAGACCGAAAGAAGTATACAGGCGT", "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC", "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]
8
5
14
